In [12]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error,accuracy_score
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load the dataset
data = pd.read_csv("yield_df.csv")
data.head()



,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,0,Albania,Maize,1990,36613,1485,121.0,16.37
1,1,Albania,Potatoes,1990,66667,1485,121.0,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485,121.0,16.37
3,3,Albania,Sorghum,1990,12500,1485,121.0,16.37
4,4,Albania,Soybeans,1990,7000,1485,121.0,16.37


In [3]:
# Encode categorical features
data['Area'] = pd.factorize(data['Area'])[0]
data['Item'] = pd.factorize(data['Item'])[0]



In [4]:
# Split data into features and target
X_static = data[['Area', 'Item', 'Year', 'pesticides_tonnes', 'avg_temp']]
y = data['hg/ha_yield']
X_time_series = data[['average_rain_fall_mm_per_year']]

# Split data into training and testing sets
X_static_train, X_static_test, y_train, y_test = train_test_split(X_static, y, test_size=0.2, random_state=42)
X_time_series_train, X_time_series_test = train_test_split(X_time_series, test_size=0.2, random_state=42)



In [5]:
# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_static_train, y_train)
rf_predictions = rf_model.predict(X_static_test)



In [6]:
# Reshape data for LSTM
X_time_series_train = np.array(X_time_series_train).reshape((X_time_series_train.shape[0], 1, 1))
X_time_series_test = np.array(X_time_series_test).reshape((X_time_series_test.shape[0], 1, 1))



In [8]:
# Build and train LSTM model
lstm_model = Sequential()
from keras.layers import Input

lstm_model.add(Input(shape=(1, 1)))
lstm_model.add(LSTM(50))

lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_time_series_train, y_train, epochs=10, batch_size=1, verbose=0)
lstm_predictions = lstm_model.predict(X_time_series_test)



96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [9]:
# Combine predictions using a simple average
combined_predictions = 0.5 * rf_predictions + 0.5 * lstm_predictions.flatten()

# Calculate accuracy metrics
mse = mean_squared_error(y_test, combined_predictions)
r2 = r2_score(y_test, combined_predictions)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 3264679217.9980593
R-squared Score: 0.553043302669405


In [13]:
mape = mean_absolute_percentage_error(y_test, combined_predictions)


In [14]:
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Model Accuracy (based on MAPE): {100 - mape * 100:.2f}%")

Mean Absolute Percentage Error: 0.47786155644909206
Model Accuracy (based on MAPE): 52.21%
